In [92]:
from numpy import genfromtxt
import os
import pandas as pd
import numpy as np
import gc

In [242]:
gc.collect()
survey_path = os.path.join(os.path.abspath(''),'surveys')
surveys = {}
for j,survey in enumerate(os.listdir(survey_path)):
    if 'Participant' in survey:
        continue
    print(survey)
    user_id = survey.split(sep=' ')[1]
    survey_file = os.path.join(survey_path,survey)
    df = pd.read_csv(survey_file,dtype='str')
    experiments = []
    for i,sl in enumerate([slice(6,17),slice(18,29), slice(30,41), slice(43,54), slice(55,66), slice(67,78)]):
        exp_df = df.iloc[sl]
        exp_df = exp_df.rename(columns={x:y for x,y in zip(df.columns,range(0,len(df.columns)))})
        exp_df = exp_df.fillna(0)
        exp_data = [exp_df.iloc[0][0], exp_df.iloc[1:-2].set_index(0), exp_df.iloc[-2:]]
        exp_data[1] = exp_data[1].replace('x',1)
        exp_data[1] = exp_data[1].iloc[:,list(range(7))]
        exp_data[1] = exp_data[1].astype(int)
        exp_data[1] = exp_data[1].dot(range(1,8))
        exp_data[1] = pd.DataFrame(exp_data[1]).transpose()
        experiments.append(exp_data)

    exp,ratings,comments = zip(*experiments)
    rate_df = pd.concat(ratings)
    rate_df_ls, rate_df_b = rate_df[:3],rate_df[3:]
    rate_df_ls = pd.DataFrame(rate_df_ls.reset_index(drop=True))
    rate_df_b = pd.DataFrame(rate_df_b.reset_index(drop=True))
    surveys[user_id] = {
                    'experiment names': exp,
                    'all ratings': rate_df.copy(),
                    'treated lightswitch ratings': rate_df_ls.copy(),
                    'treated bottle ratings': rate_df_b.copy(),
                    'comments': pd.concat(comments).copy()
                        }

User 0 - Sheet1.csv
User 1 - Sheet1.csv
User 10 - Sheet1.csv
User 2 - Sheet1.csv
User 3 - Sheet1.csv
User 4 - Sheet1.csv
User 5 - Sheet1.csv
User 6 - Sheet1.csv
User 7 - Sheet1.csv
User 9 - Sheet1.csv


In [255]:
survey_file = os.path.join(survey_path,'Participant Info - Sheet1.csv')
df = pd.read_csv(survey_file,dtype='str')
df = df[1:13][['User ID','Condition']]
df = df.set_index('User ID')
user_cond = df.to_dict()['Condition']
letter_number = {'A': 0, 'B': 1, 'C': 2}
for k,v in user_cond.items():
    order = list(v)
    order = {letter_number[letter]:i for i,letter in enumerate(order)}
    order = [order[i] for i in range(3)]
    user_cond[k] = order

In [258]:
for k in surveys.keys():
    surveys[k]['lightswitch ratings'] = surveys[k]['treated lightswitch ratings']
    surveys[k]['lightswitch ratings'] = surveys[k]['lightswitch ratings'].reindex(user_cond[k])
    surveys[k]['lightswitch ratings'] = surveys[k]['lightswitch ratings'].reset_index(drop=True)
    surveys[k]['bottle ratings'] = surveys[k]['treated bottle ratings'].reindex(user_cond[k])
    surveys[k]['bottle ratings'] = surveys[k]['bottle ratings'].reindex(user_cond[k])
    surveys[k]['bottle ratings'] = surveys[k]['bottle ratings'].reset_index(drop=True)

In [259]:
print(surveys['9']['lightswitch ratings'].transpose())

                                                    0  1  2
0                                                          
The system performed the task I wanted              5  3  4
I felt in control                                   6  2  5
The system responded to my input in the way tha...  6  1  6
The system was competent at performing tasks, e...  6  6  6
The system improved over time                       2  1  3
I improved at using the system over time            5  4  5
I always looked directly at my final target, ho...  1  2  1
I compensated for flaws in the system by changi...  7  7  7


In [264]:
ls_ratings = pd.concat([survey['lightswitch ratings'] for survey in surveys.values()])
b_ratings = pd.concat([survey['bottle ratings'] for survey in surveys.values()])
ls_ratings.index.name = 'treatment'

ls_ratings = ls_ratings.groupby(by='treatment')
ls_ratings_mean = ls_ratings.mean()
ls_ratings_mean = ls_ratings_mean.transpose()
ls_ratings_sd = ls_ratings.std()
ls_ratings_sd = ls_ratings_sd.transpose()

print(ls_ratings_mean)
print(ls_ratings_sd)

treatment                                             0    1    2
0                                                                
The system performed the task I wanted              3.2  4.1  3.3
I felt in control                                   2.9  3.5  3.1
The system responded to my input in the way tha...  3.3  3.3  3.5
The system was competent at performing tasks, e...  4.7  5.2  4.7
The system improved over time                       2.7  3.4  3.4
I improved at using the system over time            3.4  3.6  3.5
I always looked directly at my final target, ho...  2.6  3.7  3.2
I compensated for flaws in the system by changi...  5.0  5.0  5.3
treatment                                                  0         1  \
0                                                                        
The system performed the task I wanted              1.135292  1.286684   
I felt in control                                   1.370320  1.509231   
The system responded to my input in the way 